# NTUHPC Workshop: Accelerating ML Workflows with Aspire2A: The ML Part

This notebook is a continuation of the [Accelerating ML Workflows with Aspire2A](https://github.com/mrzzy/ntuhpc-workshops/tree/main/ml_aspire2a) workshop. Please refer to the README there for instructions on connecting to Aspire2A.

Upload this notebook to the JupyterHub instance on Aspire2A and run it there to ensure you're utilizing Aspire2A's powerful computing resources.

---

In this notebook, we will fine-tune [Ultralytics YOLOv11](https://github.com/ultralytics/ultralytics) model to recognize Rock-Paper-Scissors gestures using the [Roboflow Rock-Paper-Scissors dataset](https://universe.roboflow.com/roboflow-58fyf/rock-paper-scissors-sxsw).

# Download Data
Here are the instructions for downloading the data from Roboflow:

1. **Create a Roboflow Account:**
   - Visit [Roboflow](https://roboflow.com).
   - Click on **"Sign Up"** in the top right corner.
   - Fill in the required information (email, password, etc.) or sign up using Google or GitHub.
   - Complete the signup process and verify your email if needed.

2. **Navigate to the Dataset:**
   - Go to the following [Dataset URL](https://universe.roboflow.com/roboflow-58fyf/rock-paper-scissors-sxsw/dataset/14/download/yolov11).
   - This will take you directly to the dataset page for the "Rock Paper Scissors" dataset.

3. **Click on "Download":**
   - On the page, look for the **"Download"** button and click it to begin the download process.

4. **Select YOLOV11 Format & Show Download Code:**
   - Once the download prompt appears, select **"YOLOV11 format"** from the available formats.
   - Check the box labeled **"Show download code"** to generate the command for downloading.

5. **Select the "Terminal" Tab:**
   - In the download window, you’ll see multiple tabs such as **"Terminal"**, **"Python"**, and others.
   - Click on the **"Terminal"** tab to get the command for your terminal.

6. **Copy and Paste the `curl` Command:**
   - In the **"Terminal"** tab, you’ll see a `curl` command.
   - Copy the entire `curl` command and paste it down below with exclaimation sign `!` in front (eg. `!curl ....`)


In [7]:
# paste the command below: curl -L "https://universe.roboflow.com/ds/YJfJ0EONLE?key=<KEY>" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   905  100   905    0     0   2213      0 --:--:-- --:--:-- --:--:--  2212
100  227M  100  227M    0     0  14.9M      0  0:00:15  0:00:15 --:--:-- 18.0M


# Setup Dependencies
Install Dependencies

In [1]:
%pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 KB 8.7 MB/s eta 0:00:0000:0100:01
  Using cached opencv_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (63.0 MB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Using cached numpy-2.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.3 MB)
  Using cached scipy-1.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.6 MB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached torch-2.6.0-cp310-cp310-manylinux1_x86_64.whl (766.7 MB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl (26 kB)
  Using cached matplotlib-3.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.6 MB)
  Using cached pillow-11.1.0-cp310-cp310-manylinux_2_28_x86_64.whl (4.5 MB)
  Using cached torchvision-0.21.0-cp310-cp310-manylinux1_x86_64.whl (7.2 MB)
  Using cached pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
  Using cached

Import Dependencies

In [3]:
from ultralytics import YOLO
from pathlib import Path

# Fine Tune
write elaboration
- load 1

In [4]:

#TODO: replace with x
model = YOLO('yolo11n.pt')
model.train(
    data=(Path.cwd() / 'data.yaml').absolute(),
    epochs=1,                  
    imgsz=640,
    batch=16,
    device="cpu")

Ultralytics 8.3.96 🚀 Python-3.10.12 torch-2.6.0+cu124 CPU (AMD Ryzen 7 PRO 7840U w/ Radeon 780M Graphics)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/home/mrzzy/Documents/school/ntuhpc-workshops/ml_aspire2a/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=Fal

100%|████████████████████████████████████████████████████████████████████████████████| 755k/755k [00:00<00:00, 28.9MB/s]

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics


train: Scanning /home/mrzzy/Documents/school/ntuhpc-workshops/ml_aspire2a/train/labels... 6455 images, 2516 backgrounds,

train: New cache created: /home/mrzzy/Documents/school/ntuhpc-workshops/ml_aspire2a/train/labels.cache



val: Scanning /home/mrzzy/Documents/school/ntuhpc-workshops/ml_aspire2a/valid/labels... 576 images, 238 backgrounds, 0 c

val: New cache created: /home/mrzzy/Documents/school/ntuhpc-workshops/ml_aspire2a/valid/labels.cache


Plotting labels to /home/mrzzy/Documents/school/ntuhpc-workshops/runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /home/mrzzy/Documents/school/ntuhpc-workshops/runs/detect/train2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      1.386      4.227      1.589         27        640:  17%|█▋        | 70/404 [02:38<12:34,  2.


KeyboardInterrupt: 